In [1]:
import pandas as pd

## Note that this file is intended to be run with the data in a folder named DATA. Change the data_path below as needed.

In [2]:
data_path = "DATA/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
Homes_raw = pd.read_csv(data_path)
Homes_raw.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30
0,102001,0,United States,country,NaN,123328.165417,123545.139196,123814.218630,124391.341197,125055.540080,...,365372.217349,366007.694288,366471.281294,366193.660206,365661.222907,364970.033287,364347.749809,363917.639079,363688.149053,363931.687425
1,394913,1,"New York, NY",msa,NY,222096.674116,223040.459669,223992.986378,225923.174072,227921.950272,...,696533.300128,697633.014562,698948.623497,700676.336950,703154.623667,704955.084297,706457.122100,707650.653457,708454.797763,709880.464236
2,753899,2,"Los Angeles, CA",msa,CA,222620.175787,223448.605299,224552.065508,226747.580645,229148.785908,...,968173.703124,968714.163371,966675.169598,961554.657443,957217.264182,952421.630548,948047.015396,945442.656758,944366.276929,945428.022538
3,394463,3,"Chicago, IL",msa,IL,155857.310945,156001.589412,156276.370294,156959.956841,157782.229234,...,332649.038316,334015.386374,335383.265039,336242.517133,336852.936258,337148.112023,337514.741114,338367.259828,339378.680707,340732.680569
4,394514,4,"Dallas, TX",msa,TX,128023.642757,128080.664935,128146.217766,128316.451582,128540.900207,...,377150.293426,376619.688458,375829.848426,374322.685671,372189.097900,369765.587859,367394.345281,365371.532830,364005.278732,363356.229199


In [3]:
# Maybe just do top 100 for now
NUM_CITIES = 100
Homes_raw = Homes_raw.head(NUM_CITIES)

# Cleaning the data for Home Prices

### Output is melted_Homes

In [4]:
date_columns = [column for column in Homes_raw.columns if column[:4].isdigit()]

# get first few and last few to verify
print(date_columns[:4])
print(date_columns[-4:])

['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30']
['2025-06-30', '2025-07-31', '2025-08-31', '2025-09-30']


In [5]:
Homes_raw["RegionName"] = [city[:-4] for city in Homes_raw["RegionName"]]

In [6]:
# need to melt to get to work in px.line
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
melted_Homes = Homes_raw.melt(
    id_vars = ["RegionName", "StateName"],
    value_vars = date_columns,
    var_name = "Date",
    value_name = "Value"
)
print(melted_Homes.head())
print(melted_Homes.tail())

    RegionName StateName        Date          Value
0    United St       NaN  2000-01-31  123328.165417
1     New York        NY  2000-01-31  222096.674116
2  Los Angeles        CA  2000-01-31  222620.175787
3      Chicago        IL  2000-01-31  155857.310945
4       Dallas        TX  2000-01-31  128023.642757
        RegionName StateName        Date          Value
30895      Jackson        MS  2025-09-30  204224.467773
30896   Harrisburg        PA  2025-09-30  301505.765670
30897      Spokane        WA  2025-09-30  416832.678256
30898     Scranton        PA  2025-09-30  224240.674644
30899  Chattanooga        TN  2025-09-30  320484.747439


In [7]:
# turn to datetime so can do cool stuff with it, right now is string
# ONLY RUN THIS CELL ONCE
melted_Homes["Date"] = pd.to_datetime(melted_Homes["Date"])

# Creation of Interactive Linechart

In [8]:
import plotly.express as px

In [9]:
# https://plotly.com/python/line-charts/

max_y = melted_Homes['Value'].max() * 1.05
min_y = 0

fig = px.line(melted_Homes,
             x = 'Date',
             y = 'Value',
             color = 'StateName', # maybe change this later but cool to group together states
             hover_name = 'RegionName', # change hover to have cooler stuff later
             line_group = 'RegionName',
             # color_discrete_sequence = px.colors.qualitative.Set1,
             labels = {
                 'RegionName':'City',
                 'StateName':'State',
                 'Date':'Date',
                 'Value':'Average Home Price'
             },
             width = 1400,
             height = 920,
             range_y = [min_y, max_y],
             # hover_data = {'Importance':False,
             #              'Value':True,
             #              'Question':False,
             #              'ShortQ':False}
)

fig.update_layout(
    title = {'text':"City Home Prices Increase Over Time", 'xanchor':'left', 'font':{'size':24,'weight':'bold'}} # go back and change title this is just for now
)

fig.write_image("AllStatesTimeSeries.png")

fig.write_html("AllStatesTimeSeries.html")

fig.show()
### INITAL VERSION not great way too many lines and colors, could gray them out but then state information is lost. So IDK what's best to do we'll go with this for now
# because interactive = can CLICK certain states
             

C:\Users\poppy\PythonStuff\DataVis\Vis_venv\Lib\site-packages\choreographer\utils\_tmpfile.py:137: TmpDirWarning:

The temporary directory could not be deleted, execution will continue. errors: [(WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets.db/000003.log'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets.db/CURRENT'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets.db/LOCK'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets.db/LOG'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets.db/MANIFEST-000001'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpuh6y4an8/Default/Asset Store/assets

In [10]:
melted_Homes_Cali = melted_Homes[melted_Homes["StateName"] == "CA"]

In [11]:
# https://plotly.com/python/line-charts/

max_y = melted_Homes['Value'].max() * 1.05
min_y = 0

fig = px.line(melted_Homes_Cali,
             x = 'Date',
             y = 'Value',
             color = 'StateName', # maybe change this later but cool to group together states
             hover_name = 'RegionName', # change hover to have cooler stuff later
             line_group = 'RegionName',
             # color_discrete_sequence = px.colors.qualitative.Set1,
             labels = {
                 'RegionName':'City',
                 'StateName':'State',
                 'Date':'Date',
                 'Value':'Average Home Price'
             },
             width = 1200,
             height = 800,
             range_y = [min_y, max_y],
             # hover_data = {'Importance':False,
             #              'Value':True,
             #              'Question':False,
             #              'ShortQ':False}
)

fig.update_layout(
    title = {'text':"City Home Prices Increase Over Time, Especially In California", 'xanchor':'left', 'font':{'size':26}} # go back and change title this is just for now
)

fig.write_image("CaliforniaTimeSeries.png")

fig.show()
### INITAL VERSION not great way too many lines and colors, could gray them out but then state information is lost. So IDK what's best to do we'll go with this for now
# because interactive = can CLICK certain states
             

C:\Users\poppy\PythonStuff\DataVis\Vis_venv\Lib\site-packages\choreographer\utils\_tmpfile.py:137: TmpDirWarning:

The temporary directory could not be deleted, execution will continue. errors: [(WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets.db/000003.log'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets.db/CURRENT'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets.db/LOCK'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets.db/LOG'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets.db/MANIFEST-000001'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmpv_iosbgq/Default/Asset Store/assets